In [28]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI
from pydantic import BaseModel, Field, model_validator

# deployment_name = "gpt-4o-mini"
# embedding_model = "gpt-4o-mini"
# openai_api_version="2024-07-18"

system_prompt = ("""
Você é um assistente de IA que extrai dados de documentos.
Você deve extrair informações específicas e retorná-las em um formato estruturado.

INSTRUÇÕES CRÍTICAS:
- EVITE DUPLICADOS: Nunca inclua itens duplicados em qualquer lista.
- SEJA CONCISO: Mantenha cada item breve e direto ao ponto.
- VALIDE: Cada informação deve estar explicitamente declarada no texto; não faça suposições.

Por favor, extraia:
- Liste as informações do Documento

Texto do documento:
{context}

"""
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# https://cog-openai-stage.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-08-01-preview
llm = AzureChatOpenAI(azure_deployment="gpt-4o-mini", api_version="2024-08-01-preview", temperature=0.0)

In [47]:
from typing import List, Optional
from datetime import datetime, date

class LocalidadeModel(BaseModel):
    """Localização da Parte"""
    cidade: str = Field(description="Cidade da Parte")
    estado: str = Field(description="Estado da Parte")
    regiao: str = Field(description="Região do estado da Parte. Possíveis valores: S - Sul, SE - Sudeste, CO - Centro Oeste, NE - Nordeste, N - Norte")

class SecuritizadoraEmissoraModel(BaseModel):
    """Parte nomeadas e qualificadas como Securitizadora ou Emissora"""
    cnpj: str = Field(description="Número do CNPJ da Securitizadora ou Emissora. Formatos permitidos: ##.###.###/####-##")
    nome: str = Field(description="Nome da empresa ou Razão Social da Securitizadora ou Emissora")

class AgenteFiduciarioModel(BaseModel):
    """Parte nomeadas e qualificadas como Agente Fiduciário"""
    cnpj: str = Field(description="Número do CNPJ do Agente Fiduciário. Formatos permitidos: ##.###.###/####-##")
    nome: str = Field(description="Nome da empresa ou Razão Social do Agente Fiduciário")

class DevedoraModel(BaseModel):
    """Parte nomeadas e qualificadas como Devedores ou Devedoras"""
    cpf_cnpj: str = Field(description="Número do CPF ou CNPJ do Devedor. Formatos permitidos: ##.###.###/####-## ou ###.###.###-##")
    nome: str = Field(description="Nome do Devedor")
    localidade: LocalidadeModel = Field(description="Localidade do Devedor")

class CedenteModel(BaseModel):
    """Parte nomeadas e qualificadas como Cedentes"""
    cpf_cnpj: str = Field(description="Número do CPF ou CNPJ do Cedente. Formatos permitidos: ##.###.###/####-## ou ###.###.###-##")
    nome: str = Field(description="Nome do Cedente")
    localidade: LocalidadeModel = Field(description="Localidade do Devedor")

class AvalistaFiadorModel(BaseModel):
    """Parte nomeadas e qualificadas como Avalista ou Fiador"""
    cpf_cnpj: str = Field(description="Número do CPF ou CNPJ do Avalista ou Fiador. Formatos permitidos: ##.###.###/####-## ou ###.###.###-##")
    nome: str = Field(description="Nome do Avalista ou Fiador")
    localidade: LocalidadeModel = Field(description="Localidade do Devedor")

class RemuneracaoModel(BaseModel):
    """Informações sobre a remuneração do Ativo"""
    indexador: str = Field(description="Indexador da Remuneração. Valores permitidos: IPCA, DI, Pré, INCC, PTAX, IGPM",
                           examples=['IPCA', 'DI', 'Pré', 'INCC', 'PTAX', 'IGPM'])

    variacao: float = Field(description="Variação percentual acumulada do indexador da Remuneração em %", examples=[100], default=100.0)

    taxa_adicional_spread: float = Field(description="Taxa Adicional pré fixada ou Spread acrescida na Remuneração em %", examples=[1.0, 5.5, 10.0])

    periodo_taxa: str = Field(description="Periodo da Taxa Adicional. Valores permitidos: aa - para ao ano e am - para ao mês.", examples=['aa', 'am'])

class SerieModel(BaseModel):
    """Informações das Series"""
    numero_serie: str = Field(description="Número da série em sua forma apenas numérica. Caso seja uma série única, retorne UNI.",
                              examples=['1', '2', '3', 'UNI'])

    classe_serie: str = Field(description="Classe da série. Valores permitidos: Senior ou Subordinada", examples=['Senior', 'Subordinada'])

    lastro: Optional[str] = Field(description="Lastro da série",
                                examples=["Aluguel", "CCB", "CDCA", "Compra e Venda", "CPR-F", "Debênture", "Duplicata", "Financiamento Imobiliário", "Letra Financeira", "Nota Comercial", "Nota Promissória", "Outros Recebíveis", "Pro Soluto"])

    vlr_unitario_serie: float = Field(description="Valor Unitário da Série")

    qt_serie: int = Field(description="Quantidade da Série")

    vlr_serie: float = Field(description="Valor da Série")

    remuneracao: RemuneracaoModel = Field(description="Remuneração da Série")

class CoordenadorLiderModel(BaseModel):
    """Parte nomeadas e qualificadas como Coordenador Lider"""
    cnpj: str = Field(description="Número do CNPJ do Coordenador Líder. Formatos permitidos: ##.###.###/####-##")
    nome: str = Field(description="Nome da empresa ou Razão Social do Coordenador Líder")

class EstruturadorModel(BaseModel):
    """Parte nomeadas e qualificadas como Estruturador"""
    cnpj: str = Field(description="Número do CNPJ do Estruturador. Formatos permitidos: ##.###.###/####-##")
    nome: str = Field(description="Nome da empresa ou Razão Social do Estruturador")

class ResponsavelGarantiaModel(BaseModel):
    """Parte nomeadas e qualificadas como Responsável"""
    cpf_cnpj: str = Field(description="Número do CPF ou CNPJ do Responsável da Garantia. Formatos permitidos: ##.###.###/####-## ou ###.###.###-##")
    nome: str = Field(description="Nome do Responsável da Garantia")

class GarantiaModel(BaseModel):
    """Informações das Garantias"""
    garantia: str = Field(description="""Nome da Garantia.

                                        Utilize os valores possíveis:
                                        Alienação Fiduciária Imóvel,
                                        Alienação Fiduciária Maq&Eq,
                                        Penhor,
                                        Alienação Fiduciária quotas,
                                        Aval (PJ),
                                        Aval (PF),
                                        Fiança Bancária,
                                        CF Recebiveis,
                                        CF Cartão,
                                        CF Dividendos,
                                        Série Mez,
                                        Série Sub,
                                        Coobrigação,
                                        Fundo de Reserva,
                                        Fundo de Despesas,
                                        Fundo de Juros,
                                        Fundo de Obras,
                                        Outros Fundos""")

    responsavel: Optional[ResponsavelGarantiaModel] = Field(description="Responsavel Garantia")

In [42]:
class AssembleiaModel(BaseModel):
    """Informações importantes da Assembleia"""
    emissora: SecuritizadoraEmissoraModel = Field(description="Securitizadora ou Emissora")

    agente_fiduciario: AgenteFiduciarioModel = Field(description="Agente Fiduciário")

    tipo: str = Field(description="""
    Sendo CERTIFICADOS DE RECEBIVEIS DO AGRONEGOCIO = CRA;
    Sendo CERTIFICADOS DE RECEBIVEIS IMOBILIARIOS = CRI;
    Sendo DEBENTURES = DEB;
    Sendo CERTIFICADOS DE RECEBIVEIS = CR;
    Retorne se CRI, CRA, CR ou DEB. Caso não encontre, retorne ''.
    """)

    numero_emissao: str = Field(description="""
    Número da emissão em sua forma apenas numérica.
    """)

    numero_serie: List[str] = Field(description="Lista Única de Séries da Emissão em sua forma apenas numérica. Caso seja uma série única, retorne UNI")

    dt_hr_assembleia: datetime = Field(description="data e hora da assembleia")

    numero_convocacao: int = Field(description="número de convocação da assembleia nesta data")

    fl_assembleia_deliberada: bool = Field(description="A assembleia teve a Ordem do Dia deliberada?")

    fl_assembleia_nao_instalada: bool = Field(description="A assembleia não foi instalada?")

In [48]:
class TermoSecModel(BaseModel):
    """Informações importantes do Termo de Securitização"""
    emissora: SecuritizadoraEmissoraModel = Field(description="Securitizadora ou Emissora")

    agente_fiduciario: AgenteFiduciarioModel = Field(description="Agente Fiduciário")

    devedores: List[DevedoraModel] = Field(description="Lista Única de Devedores ou Devedoras")

    cedentes: List[CedenteModel] = Field(description="Lista Única de Cedentes")

    avalistas_fiadores: List[AvalistaFiadorModel] = Field(description="Lista Única de Avalistas ou Fiadores")

    coordenador_lider: Optional[CoordenadorLiderModel] = Field(description="Coordenador Lider")

    estruturador: Optional[EstruturadorModel] = Field(description="Estruturador")

    numero_aditamento: int = Field(description="Número do aditamento. Caso não seja um aditamento retorne 0", default=0)

    tipo: str = Field(description="""
    Sendo CERTIFICADOS DE RECEBIVEIS DO AGRONEGOCIO = CRA;
    Sendo CERTIFICADOS DE RECEBIVEIS IMOBILIARIOS = CRI;
    Sendo DEBENTURES = DEB;
    Sendo CERTIFICADOS DE RECEBIVEIS = CR;
    Retorne se CRI, CRA, CR ou DEB. Caso não encontre, retorne ''.
    """)
    numero_emissao: str = Field(description="Número da emissão em sua forma apenas numérica.")

    numero_serie: List[str] = Field(description="Lista Única de Séries da Emissão em sua forma apenas numérica. Caso seja uma série única, retorne UNI")

    series: List[SerieModel] = Field(description="Lista Única de Séries da Emissão. Garanta que todas as séries sejam declaradas.")

    garantias: List[GarantiaModel] = Field(description="Garantias da Emissão")

    dt_emissao: date = Field(description="data da emissão")

    dt_vencimento: date = Field(description="data do vencimento")

    numero_quorum_minimo_primeira_convocacao: str = Field(description="Número quórum mínimo de instalação de assembleia em primeira convocação com a presença de Titulares ou Investidores em Circulação. Retorne o mais simples possível.")

    numero_quorum_minimo_segunda_convocacao: str = Field(description="Número quórum mínimo de instalação de assembleia em segunda convocação com a presença de Titulares ou Investidores em Circulação. Retorne o mais simples possível.")

    vlr_unitario_emissao: float = Field(description="Valor Unitário da Emissão ou da Oferta")

    qt_total_emissao: int = Field(description="Quantidade Total da Emissão ou da Oferta")

    vlr_total_emissao: float = Field(description="Valor Total da Emissão ou da Oferta")

In [49]:
class EscrituraModel(BaseModel):
    """Informações importantes da Escritura"""
    emissora: SecuritizadoraEmissoraModel = Field(description="Securitizadora ou Emissora")

    agente_fiduciario: AgenteFiduciarioModel = Field(description="Agente Fiduciário")

    avalistas_fiadores: List[AvalistaFiadorModel] = Field(description="Lista Única de Avalistas ou Fiadores")

    coordenador_lider: Optional[CoordenadorLiderModel] = Field(description="Coordenador Lider")

    estruturador: Optional[EstruturadorModel] = Field(description="Estruturador")

    numero_aditamento: int = Field(description="Número do aditamento. Caso não seja um aditamento retorne 0", default=0)

    tipo: str = Field(description="""
    Sendo CERTIFICADOS DE RECEBIVEIS DO AGRONEGOCIO = CRA;
    Sendo CERTIFICADOS DE RECEBIVEIS IMOBILIARIOS = CRI;
    Sendo DEBENTURES = DEB;
    Sendo CERTIFICADOS DE RECEBIVEIS = CR;
    Retorne se CRI, CRA, CR ou DEB. Caso não encontre, retorne ''.
    """)

    numero_emissao: str = Field(description="""
    Número da emissão em sua forma apenas numérica.
    """)

    numero_serie: List[str] = Field(description="Lista Única de Séries da Emissão em sua forma apenas numérica. Caso seja uma série única, retorne UNI")

    series: List[SerieModel] = Field(description="Lista Única de Séries da Emissão")

    garantias: List[GarantiaModel] = Field(description="Garantias da Emissão")

    dt_emissao: date = Field(description="data da emissão")

    dt_vencimento: date = Field(description="data do vencimento")

    numero_quorum_minimo_primeira_convocacao: str = Field(description="Número quórum mínimo de instalação de assembleia em primeira convocação com a presença de Titulares ou Investidores em Circulação. Retorne o mais simples possível.")

    numero_quorum_minimo_segunda_convocacao: str = Field(description="Número quórum mínimo de instalação de assembleia em segunda convocação com a presença de Titulares ou Investidores em Circulação. Retorne o mais simples possível.")

    vlr_unitario_emissao: float = Field(description="Valor Unitário da Emissão ou da Oferta")

    qt_total_emissao: int = Field(description="Quantidade Total da Emissão ou da Oferta")

    vlr_total_emissao: float = Field(description="Valor Total da Emissão ou da Oferta")

In [33]:
def format_docs(file_path):
    f = open(file_path,"r")
    return f.read()

In [34]:
def invoke(model, file_path):
    llm_with_schema = llm.with_structured_output(model)

    chain = prompt | llm_with_schema

    result_llm = chain.invoke({"context": format_docs(file_path), "input": "Extraia as informações"})
    return result_llm

In [9]:
# FILE_PATH = '../files/BRPVSCCRI3Y2-ASS31012025V01-000829323.txt'
FILE_PATH = '../files/BRASTECRI0C3-ASS04022025V01-000830908.txt'
# FILE_PATH = '../files/23A1772203-ASS04022025V01-000831063.txt'
result = invoke(AssembleiaModel, FILE_PATH)
result.model_dump()

{'emissora': {'cnpj': '10.608.405/0001-60',
  'nome': 'Bari Securitizadora S.A.'},
 'agente_fiduciario': {'cnpj': '22.610.500/001-88',
  'nome': 'Vórtx Distribuidora de Títulos e Valores Mobiliários LTDA.'},
 'tipo': 'CRI',
 'numero_emissao': '1',
 'numero_serie': ['70'],
 'dt_hr_assembleia': datetime.datetime(2025, 2, 4, 14, 0),
 'numero_convocacao': 2,
 'fl_assembleia_deliberada': False,
 'fl_assembleia_nao_instalada': True}

## Teste Documentos de Termo de Securitização

In [52]:
FILE_PATH = '../files/831141.txt'
# FILE_PATH = '../files/24F1234491-TDS07062024V01-000675608.txt'
# FILE_PATH = '../files/797444.txt'
# FILE_PATH = '../files/018759000101010.txt'

result = invoke(TermoSecModel, FILE_PATH)
result.model_dump_json()

'{"emissora":{"cnpj":"55.085.811/0001-24","nome":"ÉXES SECURITIZADORA S.A."},"agente_fiduciario":{"cnpj":"22.610.500/0001-88","nome":"VÓRTX DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIÁRIOS LTDA."},"devedores":[{"cpf_cnpj":"02.013.649/0001-72","nome":"MADRE DE DIOS AGROPECUÁRIA E PARTICIPAÇÕES LTDA.","localidade":{"cidade":"Maracaju","estado":"Mato Grosso do Sul","regiao":"CO"}}],"cedentes":[],"avalistas_fiadores":[{"cpf_cnpj":"614.391.931-34","nome":"GRAZIELA BELLAN ALVES","localidade":{"cidade":"Maracaju","estado":"Mato Grosso do Sul","regiao":"CO"}},{"cpf_cnpj":"056.437.021-50","nome":"BRUNO BELLAN BARBOSA","localidade":{"cidade":"Maracaju","estado":"Mato Grosso do Sul","regiao":"CO"}},{"cpf_cnpj":"046.444.531-02","nome":"NATÁLIA BELLAN BARBOSA","localidade":{"cidade":"Maracaju","estado":"Mato Grosso do Sul","regiao":"CO"}},{"cpf_cnpj":"699.941.881-00","nome":"RAFAELA BELLAN","localidade":{"cidade":"Maracaju","estado":"Mato Grosso do Sul","regiao":"CO"}},{"cpf_cnpj":"45.992.343/0001-18

## Teste Documentos de Escritura de Debentures

In [53]:
FILE_PATH = "../files/676706.txt"
result = invoke(EscrituraModel, FILE_PATH)
result.model_dump_json()

'{"emissora":{"cnpj":"00.864.214/0001-06","nome":"ENERGISA S.A."},"agente_fiduciario":{"cnpj":"17.343.682/0003-08","nome":"PENTÁGONO S.A. DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIÁRIOS"},"avalistas_fiadores":[],"coordenador_lider":null,"estruturador":null,"numero_aditamento":0,"tipo":"DEB","numero_emissao":"11","numero_serie":["UNI"],"series":[{"numero_serie":"UNI","classe_serie":"Senior","lastro":"Debênture","vlr_unitario_serie":1000.0,"qt_serie":500000,"vlr_serie":500000000.0,"remuneracao":{"indexador":"IPCA","variacao":100.0,"taxa_adicional_spread":0.5,"periodo_taxa":"aa"}}],"garantias":[],"dt_emissao":"2019-04-15","dt_vencimento":"2026-04-15","numero_quorum_minimo_primeira_convocacao":"50% + 1","numero_quorum_minimo_segunda_convocacao":"1/3","vlr_unitario_emissao":1000.0,"qt_total_emissao":500000,"vlr_total_emissao":500000000.0}'